## the exploratiry data analysis

# get the data
    # data structure 
            ## data.head(); data.info(), data.col.value_counts(); 查看nan个数。
            ## 做完基础查看后应直接split data
            ## vasualization： correlation（pearson）， scatter
            ## data cleaning: fillna, imputer, normalization
    # doing datapipe and tune the model
      

In [3]:
# split data
from sklearn.model_selection import train_test_split
train_x, train_y, test_x, test_y = train_test_split(data, test_size = 0.3) 
# note: 该方法仅适用于big data，，若数据量不大，要保持representative 则使用stratified sample


In [4]:
# stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2) # n_splits 是将数据集分成 train/test pair的组数
for train_index, test_index in split.split(X, y):  
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

# 有几组代表loop循环几次
# if 只有一组的话： list(split.split(X, y)) 就获得了分组

In [13]:
# example
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4],
              [1, 2],[3, 4], [1, 2], [3, 4]])#训练数据集8*2
y = np.array([0, 0, 1, 1,0,0,1,1])#类别数据集8*1

ss=StratifiedShuffleSplit(n_splits=5,test_size=0.25,train_size=0.75,random_state=0)#分成5组，测试比例为0.25，训练比例是0.75

for train_index, test_index in ss.split(X, y):  #有5组，每一组有train， test
   print("TRAIN:", train_index, "TEST:", test_index)#获得索引值
   X_train, X_test = X[train_index], X[test_index]#训练集对应的值
   y_train, y_test = y[train_index], y[test_index]#类别集对应的值

# # 可以用来cross_validation

TRAIN: [5 2 6 4 1 3] TEST: [7 0]
TRAIN: [4 3 5 2 7 1] TEST: [6 0]
TRAIN: [7 1 6 2 0 4] TEST: [5 3]
TRAIN: [3 6 4 7 0 5] TEST: [1 2]
TRAIN: [3 4 1 7 2 0] TEST: [6 5]


In [ ]:
#  vasualization
    ## scatter plot 
df.plot(kind = 'scatter', x = '', y= '', alpha = 0.1) # alpha: high density data points 就会变深
# 一张图上有时候可以表达两个指标：p57
housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude', alpha = 0.4, 
            s = housing.population/100, label = 'population', figsize = (10,7),
            c = 'price', cmap = plt.get_cmap('jet'), colorbar = True)
plt.legend()

# s: 每一个点的半径大小和人口数量有关， label：是人口（右上角的）， c:颜色和房价有关， cmap： 用什么颜色图谱表示, colorbar
    # 可以看房价和人口多少，地点有没有关系


# corelation
    ## 第一种，直接算correlation 值： df.corr(): 会得到一个matrix
pearson_matrix = df.corr()
    ## 第二种，画图
from pandas.plotting import scatter_matrix
attributes = ['', '', '']
scatter_matrix(df[attributes], figsize = (12,8))


## impute package
https://scikit-learn.org/stable/modules/impute.html

In [ ]:
from sklearn.impute import SimpleImputer 
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.tree import DecisionTreeClassifier

# 只有连续变量， indicator必须要用 FeatureUnion
transformer = FeatureUnion(transformer_list=
                           [('features', SimpleImputer(strategy='mean')),('indicators', MissingIndicator())])


# fit the model
clf = Pipeline([('trans',transformer), ('class', DecisionTreeClassifier())])
## 也可以写成：
clf = make_pipeline(transformer, DecisionTreeClassifier())
clf = clf.fit(X_train, y_train)
results = clf.predict(X_test)
results.shape



# normalization
    ## minmaxscalar:(feature_range)
    ## standardscaler: standardazation: does not bound values to a specific range, less affected by outliers
https://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    
## 到底哪些模型需要normalization呢？

概率模型（树形模型）不需要归一化，因为它们不关心变量的值，而是关心变量的分布和变量之间的条件概率，如决策树、RF。而像Adaboost、SVM、LR、Knn、KMeans之类的最优化问题就需要归一化。
    

## 对连续变量和离散变量一起写数据预处理pipeline

In [18]:
from sklearn.compose import ColumnTransformer
num_attribs = [一些连续变量]
cat_attribs = [分类变量]
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
    
])

new = full_pipeline.fit_transform(old) 

## 对连续变量和离散变量一起写模型pipeline

https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

In [23]:
# Author: Pedro Morales <part.morales@gmail.com>
#
# License: BSD 3 clause

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

np.random.seed(0)

# Load data from https://www.openml.org/d/40945
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# Alternatively X and y can be obtained directly from the frame attribute:
# X = titanic.frame.drop('survived', axis=1)
# y = titanic.frame['survived']

# We will train our classifier with the following features:
# Numeric Features:
# - age: float.
# - fare: float.
# Categorical Features:
# - embarked: categories encoded as strings {'C', 'S', 'Q'}.
# - sex: categories encoded as strings {'female', 'male'}.
# - pclass: ordinal integers {1, 2, 3}.

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare'] # 连续
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass'] #分类
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)]) # 合在一起，de

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.790


## fine tune the model
### GridSearch

In [26]:
# Author: Pedro Morales <part.morales@gmail.com>
#
# License: BSD 3 clause

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

np.random.seed(0)

# Load data from https://www.openml.org/d/40945
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

# Alternatively X and y can be obtained directly from the frame attribute:
# X = titanic.frame.drop('survived', axis=1)
# y = titanic.frame['survived']

# We will train our classifier with the following features:
# Numeric Features:
# - age: float.
# - fare: float.
# Categorical Features:
# - embarked: categories encoded as strings {'C', 'S', 'Q'}.
# - sex: categories encoded as strings {'female', 'male'}.
# - pclass: ordinal integers {1, 2, 3}.

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare'] 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy=param_grid['preprocessor__num__imputer__strategy'])),   # 要超参数放进去
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass'] #分类
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)]) # 合在一起，de

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

############################################################################

from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    #'classifier__C': [0.1, 1.0, 10, 100],
}

grid_search = GridSearchCV(clf, param_grid, cv=10) # clf 是上面的pipeline



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
grid_search.fit(X_train, y_train)
grid_search.cv_results_ # 一个dict，转成df看，找到tune完的结果
print(("best logistic regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

# 得到最优参数后，重新train？不需要直接给结果： 
    ## predict(X) Call predict on the estimator with the best found parameters.
grid_search.predict(X_test)

best logistic regression from grid search: 0.790


array(['0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '1', '1', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1',
       '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0',
       '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1',
       '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1',
       '1', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1',
       '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0',
       '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0',
       '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '0',
       '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0',
       '1', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0',
       '0', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1

## randomized search

In [27]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    #'classifier__C': [0.1, 1.0, 10, 100],
}

grid_search = RandomizedSearchCV(clf, param_grid, cv=10) # clf 是上面的pipeline



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
grid_search.fit(X_train, y_train)
grid_search.cv_results_ # 一个dict，转成df看，找到tune完的结果
print(("best logistic regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

# 得到最优参数后，重新train？不需要直接给结果： 
    ## predict(X) Call predict on the estimator with the best found parameters.
grid_search.predict(X_test)

/opt/anaconda3/envs/python37/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


best logistic regression from grid search: 0.794


array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1',
       '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1',
       '1', '0', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0',
       '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1',
       '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1',
       '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0',
       '0', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1',
       '0', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0',
       '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1',
       '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0',
       '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0

## Ensemble methods
